In [22]:
import json, re
import requests
from urlextract import URLExtract
import sys, gzip


utid = 'awarden9'
base = {
    'model': 'https://huggingface.co/',
    'data': 'https://huggingface.co/datasets/',
    'source': 'https://'
}

post = '/raw/main/README.md'
postGH = 'blob/master/README.md'  

extU = URLExtract()
DOIpattern = r'\b(10\.\d{4,9}\/[-._;()/:A-Z0-9]+)\b/i'


def extractURLs(content):
    res = extU.find_urls (content)
    return res

def extractDOIs(content):
    res = re.findall (DOIpattern, content)
    return res

def run (tp):
    with open(f"input/{utid}_{tp}", 'r') as f:
        for line in f:
            line = line.strip()
            post0 = postGH if tp == 'source' else post
            id = line.split(';')[1] if tp == 'source' else line
            
            url = f"{base[tp]}{id}{post0}"
            r = requests.get(url)
            if r.status_code != 200:
                continue
            
            content = r.text.replace('\n', ' ')
            urls = extractURLs(content)
            dois = extractDOIs(content)
            
            res = {
                'id': id,
                'type': tp,
                'url': url,
                'content': content,
                'links': urls,
                'dois': dois,
            }
            yield res

with gzip.open(f"output/{utid}.json.gz", 'wt', encoding='utf-8') as fo:
    for tp in ['model', 'data', 'source']:
        for entry in run (tp):
            json.dump(entry, fo)
            fo.write('\n')